In [2]:
import config
import os
import torch
import json
import pickle
from custom_data import CustomDataset
from tokenizer import Tokenizer
from torch.utils.data import DataLoader
from train import evaluate
from torch import nn
import numpy as np
import plotly.express  as px

/home/miniconda3/envs/pytorch_p39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
check_path = "/home/Ravikumar/Developer/Learning-NLP-with-PyTorch/Chapter - 7 : Detailed Understanding of Multi-Label Classification/dataset/checkpoints/transformer_multilabel-classification_added_expansion_layer_bs_64_lr_0.0001.pt"
check_path1 = "/home/Ravikumar/Developer/Learning-NLP-with-PyTorch/Chapter - 7 : Detailed Understanding of Multi-Label Classification/dataset/checkpoints/transformer_multilabel-classification_added_expansion_layer_pos_weight_bs_64_lr_0.0001.pt"
checkpoints = torch.load(check_path1)

In [13]:
args_dict = checkpoints['params']

In [14]:
# args_dict =  {k:v for k,v in config.__dict__.items() if "__" not in k}

vocab = json.load(open(args_dict["VOCAB_DIR"],"r"))
word_to_token = {k:int(v) for k,v in vocab["vocabs"].items()}
args_dict["N_WORDS"] = len(word_to_token)

In [15]:
if os.path.isfile(args_dict["CHECKPOINT_NAME"]):
    checkpoints = torch.load(args_dict["CHECKPOINT_NAME"])


In [16]:
from model import ClassifierModel
model = ClassifierModel(args_dict=args_dict).to(args_dict['DEVICE'])
model.load_state_dict(checkpoints["model_state_dict"])

<All keys matched successfully>

In [17]:
splitted_data = pickle.load(open(args_dict["SPLIT_DATA_DIR"],"rb"))
valid_dataset = splitted_data["valid_dataset"][args_dict["LABELS"] +["pairs"]]
train_dataset = splitted_data["train_dataset"][args_dict["LABELS"] +["pairs"]]

In [ ]:
train_dataset

In [21]:
train_dataset.iloc[:,:-1].sum(0)

Computer Science        6874
Physics                 4787
Mathematics             4503
Statistics              4189
Quantitative Biology     461
Quantitative Finance     205
dtype: int64

In [23]:
len(train_dataset)

16777

In [24]:
train_dataset.iloc[:,:-1].sum(0).max()

6874

In [25]:

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Train_True', y=train_dataset.iloc[:,:-1].sum(0)),
    go.Bar(name='Val_True', y=valid_dataset.iloc[:,:-1].sum(0))
])
fig.update_layout(barmode='group')
fig.show()

In [18]:

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Train_True', y=train_dataset.iloc[:,:-1].sum(0)/train_dataset.iloc[:,:-1].sum(0).max()),
    go.Bar(name='Val_True', y=valid_dataset.iloc[:,:-1].sum(0)/valid_dataset.iloc[:,:-1].sum(0).max())
])
fig.update_layout(barmode='group')
fig.show()

In [20]:
tokenizer_obj = Tokenizer("topic-modelling-research-articles")
validation_set = CustomDataset(
    valid_dataset,
    tokenizer=tokenizer_obj,
    args=args_dict,
)

validation_dataloader = DataLoader(
    validation_set,
    batch_size = args_dict["BATCH_SIZE"],
    drop_last = True)

In [21]:
loss_fn = nn.BCEWithLogitsLoss()
test_loss,val_all_pred,val_all_true,val_metrics = evaluate(model,validation_dataloader,args_dict,loss_fn)

 40%|████      | 26/65 [00:05<00:08,  4.41it/s]


KeyboardInterrupt: 

In [5]:
pred = np.array(val_all_pred).sum(axis = 0)
true = np.array(val_all_true).sum(axis = 0)
pred,true

NameError: name 'val_all_pred' is not defined

In [6]:
px.bar(pred)
px.bar(true)

NameError: name 'pred' is not defined

In [81]:
true.max()

1705.0

In [80]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Preds', y=pred/true.max()),
    go.Bar(name='True', y=true/true.max())
])
fig.update_layout(barmode='group')
fig.show()

In [88]:
next(iter(validation_dataloader))

{'input_ids': tensor([[33522, 15477, 19164,  ...,     0,     0,     0],
         [20690, 38356, 31234,  ...,     0,     0,     0],
         [20576, 24924, 11142,  ...,     0,     0,     0],
         ...,
         [40846, 24745, 22540,  ...,     0,     0,     0],
         [17988, 31559, 29457,  ...,     0,     0,     0],
         [36842, 13471, 25949,  ...,     0,     0,     0]]),
 'target': tensor([[0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 1., 0., 0.],
         [1., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [1.

In [24]:
torch.tensor(validation_dataloader.dataset.pos_weight)

tensor([ 1.4390,  2.4217,  2.7623,  3.1249, 32.2937, 94.3409],
       dtype=torch.float64)

: 